# Type of DFT/MD calculations
NOTE: DFT/MD shared the same calculation types, so the following information is applicable to both DFT and MD calculations. DFT and MD only share many common concepts, the only difference is that
- MD used a *predefined force field* to calculate the forces between atoms, 
- while DFT used the *quantum mechanical methods* to calculate the forces between atoms.


Need to differentiate between 3 types of calculations: `single-point` energy calculation, `geometry optimization` and `molecular dynamics`

1. **Single-Point Energy Calculation**: 

   This is the simplest type of calculation. You provide the atomic positions and the program calculates the total energy of the system without changing the positions of the atoms. This is useful if you already know the optimized geometry of your system and just want to calculate properties at that geometry, such as the total energy, electronic structure, or other properties that can be derived from the wavefunction.

2. **Geometry Optimization**: 

   In this type of calculation, the program adjusts the positions of the atoms in order to find the geometry that minimizes the total energy of the system. This is typically the first step in a computational study, because many properties are dependent on the atomic positions. The optimization algorithm adjusts the atomic positions in each step based on the forces acting on the atoms, and continues until the forces are below a certain threshold, indicating that the geometry is optimized.
   
3. **Molecular Dynamics (MD)**: 

   In an MD simulation, the program calculates the forces on the atoms and uses these to propagate the atomic positions in time according to Newton's laws of motion. This allows the system to explore a range of geometries and can be used to simulate temperature and pressure effects, study dynamic processes, or sample configurations for further study. In contrast to a geometry optimization, which finds the minimum energy configuration, an MD simulation can explore higher-energy configurations and provides information on the dynamics of the system.

## ASE-MD simulation 
REF:
- https://wiki.fysik.dtu.dk/ase/ase/md.html#module-ase.md
- https://wiki.fysik.dtu.dk/ase/tutorials/md/md.html
- https://docs.matlantis.com/atomistic-simulation-tutorial/en/6_1_md-nve.html

Once the structure has been created:
- the *initial velocity* of each atom is given by a velocity distribution corresponding to the specified temperature [Maxwell–Boltzmann distribution](https://en.wikipedia.org/wiki/Maxwell%E2%80%93Boltzmann_distribution).
- Since the initial velocity given by this method has *arbitrary momentum of the whole system*, there is a possibility that the whole system may drift. Therefore, after giving the initial velocity by `MaxwellBoltzmannDistribution`, we set the momentum of the entire system to zero by the `Stationary` method and fix the coordinates of the center of mass. This is important not only for the NVE case, but also for simulations involving temperature and pressure control that will follow.

```py
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution,Stationary

# Set the momenta corresponding to the given "temperature"
MaxwellBoltzmannDistribution(atoms, temperature_K=temperature,force_temp=True)
Stationary(atoms)  # Set zero total momentum to avoid drifting
```

### NVE ensemble
The [ensemble](https://en.wikipedia.org/wiki/Ensemble_(mathematical_physics)) here refers to the distribution of states of system in the concept of statistical mechanics.

microcanonical ensemble (NVE) is a closed system in which the number of particles, the volume, and the total energy are conserved.
- the system evolves naturally in time according to the equations of motion of classical mechanics. 
- Since there is no external force, the energy conservation law is obeyed and the total energy is kept constant within the numerical error. 
- The temperature decreases fairly quickly, and the kinetic and potential energies follows the [equipartition theorem](https://en.wikipedia.org/wiki/Equipartition_theorem), and the temperature eventually settles to about half the initial temperature

REF:
- https://docs.matlantis.com/atomistic-simulation-tutorial/en/6_1_md-nve.html


```python
from ase.md.verlet import VelocityVerlet
# Define the MD dynamics class object
dyn = VelocityVerlet(atoms, 1 * units.fs)  # 1 fs time step
dyn.run(atoms, steps=1000)
```

### NVT ensemble
canonical ensemble (NVT ensemble) keep the temperature and volume constant during the simulation.

There are several methods (thermostats, heat-paths) to control the temperature in MD simulations, some implemented in ASE:
- Berendsen: simple, but it changes the velocity distribution of atoms in the entire system uniformly, which can result in *unnatural phenomena*.
- Langevin: applying random forces to individual atoms, which are appropriately sampled using a statistical method.  It is not suitable for precise trajectory analysis because it does not reproduce the same trajectories even after repeated calculations due to its statistical nature.
- Nosé-Hoover: is one of the most universally used temperature control methods. But unstable in some cases.
- Noose-Hoover-Chain: an extension of the Nosé-Hoover method, which is more stable and accurate than the Nosé-Hoover method.

Conclusions: should use the *Nosé-Hoover-Chain* method for temperature control in MD simulations.

REF:
- https://docs.matlantis.com/atomistic-simulation-tutorial/en/6_2_md-nvt.html
- issue: https://gitlab.com/ase/ase/-/merge_requests/3508/diffs
- code: 


```py
from ase.md.nose_hoover_chain import NoseHooverChainNVT
dyn = NoseHooverChainNVT(
        atoms,
        timestep=timestep,
        temperature_K=temperature_K,
        tdamp=100 * timestep,
        tchain=tchain,
    )
dyn.run(steps=1000)
```

### NPT ensemble
isothermal-isobaric ensemble (NPT ensemble) keep the temperature, pressure, and volume constant during the simulation.

in ASE only two NPT methods are implemented:
- Parrinello-Rahman: use Noose-Hoover thermostat and Parrinello-Rahman barostat.
    - use parameter `pfactor` to control the pressure, which is different with `pdamp` in Nose-Hoover-Chain method. 
- NPTBerendsen/InhomogeneousBerendsen: use Berendsen thermostat and Berendsen barostat.
- Nose-Hoover-Chain: use Noose-Hoover-Chain thermostat and barostat 
    - Iso-Nose-Hoover-Chain [implemented in this issue](https://gitlab.com/ase/ase/-/merge_requests/3550).

REF:
- https://docs.matlantis.com/atomistic-simulation-tutorial/en/6_3_md-npt.html

```python
from ase.md.npt import NPT
dyn = NPT(
        atoms,
        timestep=timestep,
        temperature_K=temperature_K,
        externalstress=0.0,
        ttime=100 * timestep,
        pfactor=1e6,  # None for NVT
        mask=None,
    )
dyn.run(steps=1000)
```

#### Note
- `NPT` class in ASE requires structure to have box is an *upper triangular matrix*.
- `Noose-Hoover-Chain` method *does not* require upper triangular matrix.

# ASE optimization
REF: https://wiki.fysik.dtu.dk/ase/ase/optimize.html

Some notes:
- The optimization can be restarted by loading trajectory file `opt.replay_trajectory('history.traj')`. See more: https://wiki.fysik.dtu.dk/ase/ase/optimize.html
- If using filter, `ExpCellFilter` nust be use after set calculator `atoms.calc`

## Forces matrix with vs. without `filter()`
The `filter()` method in ASE is used to *relax the cell* during optimization. When `filter()` is applied, the forces on the cell boundaries are added as additional degrees of freedom to the optimization [(equivalent to add more 3 “atoms”)](https://wiki.fysik.dtu.dk/ase/ase/filters.html#ase.filters.ExpCellFilter). 

In optimization, the optimizer minimizes the maximum force.
- If no `filter()`, the maximum force is the maximum force of *atoms only*. The forces to optimize are N*3 
- If `filter()`, the maximum force is the maximum force of *atoms and cell boundaries*, the optimizer has to optimize the forces on atoms and the forces on cell boundaries simultaneously. The forces to optimize are (N+3)*3

Example system with 2 atoms, the `filter()` method add *3 more rows to the force matrix*. The forces on atoms are very small, but the forces on cell boundaries are large. 
```log
[[3.232892355146985e-33, -6.466788719144503e-28, -2.255405052076729e-27], 
[-3.23289235340891e-33, 2.2554050520767286e-27, 6.466788719144512e-28], 

[-0.03236528299346515, -5.3176216729227915e-05, -5.3176216729235274e-05], 
[-6.767443558001566e-05, 0.023388583117855567, 0.0063938751340083776], 
[-6.767443558000313e-05, 0.006393875134008386, 0.02338858311785556]]
```

Example: See the example below for more details about forces in optimization.
- `fmax_log` in logfile is the maximum force viewed by the optimizer
```py
#####ANCHOR Relax structure
atoms.calc = calc
atoms_filter = FrechetCellFilter(atoms, mask=[1, 1, 1, 0, 0, 0])
opt = BFGS(atoms_filter)

def print_dynamic(atoms=atoms, filename="dyn_properties.txt"):
    step = opt.nsteps
    ### Props on `atoms` object
    energy = atoms.get_potential_energy()
    forces = atoms.get_forces()
    fmax = sqrt((forces**2).sum(axis=1).max())
    stress = atoms.get_stress()  # 6-vector in Voigt notation
    aveStress = sqrt((stress**2).sum(axis=0).mean())
    ### Props on `atoms_filter` object
    energy_filter = atoms_filter.get_potential_energy()
    forces_filter = atoms_filter.get_forces()
    fmax_filter = sqrt((forces_filter**2).sum(axis=1).max())

    if not Path(filename).exists():
        with paropen(filename, "w") as f:
            f.write("step energy fmax aveStress energy_filter fmax_filter \n")
    with paropen(filename, "a") as f:
        f.write(
            f"{step} {energy:.7f} {fmax:.7f} {aveStress:.7f} {energy_filter:.7f} {fmax_filter:.7f}\n"
        )

opt.attach(print_dynamic, interval=1)
opt.run(fmax=0.01)
```

- If optimize atoms only, ie., `opt = BFGS(atoms)`, the properties during optimization are:
    - `fmax_log` in this case is the maximum force of atoms `fmax`
    - Because the cell is not relaxed, so `fmax_filter` is unchanged (force on cell boundaries is not optimized)
```sh
step	energy	fmax	aveStress	energy_filter	fmax_filter	fmax
0	-28.2984042	0.1640364	0.0240364	-28.2990295	0.1640364	0.164036
1	-28.2998351	0.1311988	0.0241873	-28.3004618	0.1629209	0.131199
2	-28.3026799	0.0546654	0.0243134	-28.3033111	0.1638302	0.054665
3	-28.302979	0.0504313	0.0243758	-28.3036112	0.1642599	0.050431
4	-28.303317	0.0549254	0.0243898	-28.3039499	0.1643315	0.054925
5	-28.3036777	0.0398742	0.0243907	-28.3043107	0.1642861	0.039874
6	-28.3039163	0.0265531	0.0244998	-28.3045482	0.1650522	0.026553
7	-28.3039824	0.0161706	0.0244443	-28.3046147	0.1646465	0.016171
8	-28.3039985	0.0147324	0.0244203	-28.304631	0.164481	0.014732
9	-28.3040181	0.0172235	0.0244181	-28.3046508	0.1644513	0.017223
10	-28.3040423	0.0082473	0.0244198	-28.3046751	0.1644544	0.008247
```

- If optimize atoms and cell boundaries, ie., `opt = BFGS(atoms_filter)`, the properties during optimization are:
    - `fmax_log` in this case is the maximum force of atoms and cell boundaries `fmax_filter`, and will slightly different with `fmax` of atoms.
    - Because the cell is relaxed, so `fmax_filter` is reduced
```sh
step	energy	fmax	aveStress	energy_filter	fmax_filter	fmax
0	-28.2984042	0.1640364	0.0240364	-28.2990295	0.1640364	0.164036
1	-28.2915354	0.1546100	0.0235672	-28.2921599	0.1590883	0.159088
2	-28.1915797	0.4433223	0.0110966	-28.1921873	0.4406105	0.440611
3	-28.2743333	0.0923216	0.0105981	-28.2749595	0.0920909	0.092091
4	-28.2732457	0.0870674	0.0110138	-28.2738717	0.0868412	0.086841
5	-28.2184204	0.2238904	0.0083471	-28.2190418	0.2225657	0.222566
6	-28.2553818	0.0326038	0.0028914	-28.2560054	0.0324713	0.032471
7	-28.2551177	0.0292529	0.0019618	-28.2557409	0.0291281	0.029128
8	-28.2418554	0.0226584	0.0019369	-28.2424773	0.0225475	0.022548
9	-28.2475649	0.0210898	0.0012095	-28.2481871	0.0209894	0.020989
```

## Save value: `atoms` vs. `atoms_filter`
IMPORTANT: As above this discussion, the `atoms_filter` object only differs from `atoms` in the forces degree of freedoms (`atoms_filter` has 3 more rows for cell boundaries). These extra DOFs only used for optimization. All other properties are the same (atomic positions/forces, energy, stress, etc.). 

So, we just need to record the properties of `atoms` object.

# Energy calculation

## About force consistent energies
- See discussion in [ASE issue 1379](https://gitlab.com/ase/ase/-/issues/1379). Still in discussion. Just use `get_potential_energy(force_consistent=False)` as default to get the energy.

ASE currently knows 2 energy definitions rooted in DFT terminology:
- `free_energy` is an electronic free energy which includes smearing. Since the smearing does not correspond to a physical temperature, and often smearing schemes other than Fermi-Dirac are used, the value of `free_energy` has no physical meaning. Yet in many DFT codes, forces are consistent with this energy.
- `energy` is the free energy extrapolated to 0K to remove the smearing which leads to a "corrected DFT total energy".

Depending on context, both can be returned by `get_potential_energy()`.

# Units conversion
In the Atomic Simulation Environment (ASE), quantities (e.g., lengths, energies, forces) are represented in [*ASE's internal units*](https://wiki.fysik.dtu.dk/ase/ase/units.html). If you're working with a different set of units (like SI units or other physical unit systems), you'll need to convert between your units and ASE's internal units.

ASE Internal Units
- Length: Ångström (Å) = 10^-10 meters
- Energy: electronvolt (eV)
- Force: eV/Å
- Time: fs (femtoseconds)

Conversion ford/back between ASE's internal units and SI units:
```python
from ase.units import GPa

pressure_in_gpa = pressure_in_ev_per_angstrom3 * GPa
pressure_in_ev_per_angstrom3 = pressure_in_gpa / GPa
```